In [1]:
import torch
import torch_geometric as pyg

In [2]:
torch.tensor([1,2,3])

tensor([1, 2, 3])

In [3]:
torch.tensor("hallo")

TypeError: new(): invalid data type 'str'

In [10]:
t1 = pyg.data.Data(x=torch.tensor([[1,2,3],[4,5,6]]), edge_index=torch.tensor([[0], [1]]), tree="hello")
print(t1)
print(t1.x)
print(t1.tree)
t1 = t1.to('cuda')
print(t1)
print(t1.x)
print(t1.tree)

Data(x=[2, 3], edge_index=[2, 1], tree='hello')
tensor([[1, 2, 3],
        [4, 5, 6]])
hello
Data(x=[2, 3], edge_index=[2, 1], tree='hello')
tensor([[1, 2, 3],
        [4, 5, 6]], device='cuda:0')
hello


In [15]:
b1 = pyg.data.Batch.from_data_list([t1, t1, t1])
b1

DataBatch(x=[6, 3], edge_index=[2, 3], tree=[3], batch=[6], ptr=[4])

In [16]:
print(b1.tree)
print(b1.x)

['hello', 'hello', 'hello']
tensor([[1, 2, 3],
        [4, 5, 6],
        [1, 2, 3],
        [4, 5, 6],
        [1, 2, 3],
        [4, 5, 6]], device='cuda:0')
